In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE61322"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE61322"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE61322.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE61322.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE61322.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Mutation of senataxin alters disease-specific transcriptional networks in patients with ataxia with oculomotor apraxia type 2 [03_AOA2_patient_blood_2011]"
!Series_summary	"Senataxin, encoded by the SETX gene, contributes to multiple aspects of gene expression, including transcription and RNA processing. Mutations in SETX cause the recessive disorder ataxia with oculomotor apraxia type 2 (AOA2) and a dominant juvenile form of amyotrophic lateral sclerosis (ALS4). To assess the functional role of senataxin in disease, we examined differential gene expression in AOA2 patient fibroblasts, identifying a core set of genes showing altered expression by microarray and RNA-sequencing. To determine whether AOA2 and ALS4 mutations differentially affect gene expression, we overexpressed disease-specific SETX mutations in senataxin-haploinsufficient fibroblasts and observed changes in distinct sets of genes. This implicates mutation-specific alterations of se

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarray analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify the keys for trait, age, and gender
# From the Sample Characteristics, we can see that:
# Key 0 has ['diagnosis: carrier', 'diagnosis: affected'] which can be used for the trait
trait_row = 0
# Age is not explicitly available in the provided sample characteristics
age_row = None
# Gender is not explicitly available in the provided sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert trait value to binary format."""
    if value is None or pd.isna(value):
        return None
    
    if isinstance(value, str):
        value = value.lower().strip()
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "affected" in value:
            return 1  # For affected patients (AOA2)
        elif "carrier" in value:
            return 0  # For carriers/controls
    
    return None

def convert_age(value: str) -> float:
    """Convert age value to continuous format."""
    # Age data is not available, but this function is defined for completeness
    return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary format."""
    # Gender data is not available, but this function is defined for completeness
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if is_trait_available:
    # Load clinical data
    try:
        files = os.listdir(in_cohort_dir)
        clinical_data_file = None
        for file in files:
            if file.endswith("_series_matrix.txt"):
                clinical_data_file = os.path.join(in_cohort_dir, file)
                break
        
        if clinical_data_file:
            # Load the file to extract sample characteristics
            sample_data = []
            with open(clinical_data_file, 'r') as f:
                for line in f:
                    if line.startswith('!Sample_char') or line.startswith('!Sample_characteristics'):
                        parts = line.strip().split('\t')
                        if len(parts) > 1:
                            sample_data.append(parts[1:])
            
            # Create clinical dataframe if data is found
            if sample_data:
                # Transpose the data to have samples as columns
                clinical_df = pd.DataFrame(sample_data)
                
                # Extract clinical features using the library function
                selected_clinical_df = geo_select_clinical_features(
                    clinical_df=clinical_df,
                    trait=trait,
                    trait_row=trait_row,
                    convert_trait=convert_trait,
                    age_row=age_row,
                    convert_age=convert_age,
                    gender_row=gender_row,
                    convert_gender=convert_gender
                )
                
                # Preview the selected clinical features
                preview = preview_df(selected_clinical_df)
                print("Preview of selected clinical features:")
                print(preview)
                
                # Create directory if it doesn't exist
                os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
                
                # Save clinical data
                selected_clinical_df.to_csv(out_clinical_data_file)
                print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {str(e)}")
        # Even if extraction fails, we've already recorded the trait availability


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')

Gene data dimensions: 24525 genes × 33 samples


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols
# These are microarray probe identifiers from Illumina's BeadArray technology
# They need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1722532', 'ILMN_1708805', 'ILMN_1672526', 'ILMN_1703284', 'ILMN_2185604'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_25544', 'ILMN_10519', 'ILMN_17234', 'ILMN_502', 'ILMN_19244'], 'Transcript': ['ILMN_25544', 'ILMN_10519', 'ILMN_17234', 'ILMN_502', 'ILMN_19244'], 'ILMN_Gene': ['JMJD1A', 'NCOA3', 'LOC389834', 'SPIRE2', 'C17ORF77'], 'Source_Reference_ID': ['NM_018433.3', 'NM_006534.2', 'NM_001013655.1', 'NM_032451.1', 'NM_152460.2'], 'RefSeq_ID': ['NM_018433.3', 'NM_006534.2', 'NM_001013655.1', 'NM_032451.1', 'NM_152460.2'], 'Entrez_Gene_ID': [55818.0, 8202.0, 389834.0, 84501.0, 146723.0], 'GI': [46358420.0, 32307123.0, 61966764.0, 55749599.0, 48255961.0], 'Accession': ['NM_018433.3', 'NM_006534.2', 'NM_001013655.1', 'NM_032451.1', 'NM_152460.2'], 'Symbol': ['JMJD1A', 'NCOA3', 'LOC389834', 'SPIRE2', 'C17orf77'], 'Prot

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe IDs and gene symbols
# From the annotation preview, we can see:
# - 'ID' column contains the Illumina probe IDs matching the gene expression data index
# - 'Symbol' column contains the corresponding gene symbols

# 2. Get the gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print a preview of the mapping
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the resulting gene expression data
print(f"\nConverted gene expression data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 10 gene symbols:")
print(gene_data.index[:10])


Gene mapping preview (first 5 rows):
             ID       Gene
0  ILMN_1722532     JMJD1A
1  ILMN_1708805      NCOA3
2  ILMN_1672526  LOC389834
3  ILMN_1703284     SPIRE2
4  ILMN_2185604   C17orf77

Converted gene expression data dimensions: 17824 genes × 33 samples

First 10 gene symbols:
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (16856, 33)
First 5 gene symbols after normalization: Index(['A1BG', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE61322.csv
Sample IDs in clinical data:
Index(['!Sample_geo_accession', 'GSM1502059', 'GSM1502060', 'GSM1502061',
       'GSM1502062'],
      dtype='object') ...
Sample IDs in gene expression data:
Index(['GSM1502059', 'GSM1502060', 'GSM1502061', 'GSM1502062', 'GSM1502063'], dtype='object') ...
Clinical data shape: (1, 33)
Clinical data preview: {'GSM1502059': [0.0], 'GSM1502060': [0.0], 'GSM1502061': [0.0], 'GSM1502062': [1.0], 'GSM1502063': [0.0], 'GSM1502064': [1.0], 'GSM1502065': [1.0], 'GSM1502066': [1.0], 'GSM1502067': [1.0], 'GSM1502068': [0.0], 'GSM1502069': [1.0], 'GSM1502070': [1.0], 'GSM1502071': [1.0], 'GSM1502072': [0.0], 'GSM1502073': [1.0], 'GSM1502074': [0.0], 'GSM1502075': [1.0], 'GSM1502076': [0.0], 'GSM1502077': [1.0], 'GSM1502078': [1.0], 'GSM1502079': [0.0], 'GSM1502080': [0.0], 'GSM1502081': [0.0], 'GSM1502082': [0.0], 'GSM1502083': [0.0], 'GSM1502084': [0.0], 'GSM15020

Data shape after handling missing values: (33, 16857)
For the feature 'Amyotrophic_Lateral_Sclerosis', the least common label is '1.0' with 15 occurrences. This represents 45.45% of the dataset.
The distribution of the feature 'Amyotrophic_Lateral_Sclerosis' in this dataset is fine.

Data shape after removing biased features: (33, 16857)


Linked data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE61322.csv
